![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Advanced Futures Trading Strategies

"Advanced Futures Trading Strategies" by Robert Carver lays the foundation for building a systematic trading system.

Although the book focuses on futures trading, its concepts are foundational and applicable to other financial instruments.

This notebook replicates the book's initial strategies, emphasizing data processing, metric building, and position size management with risk targeting.

# Data aquisition

In [94]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from QuantConnect.Data.UniverseSelection import *
from helper import calculate_and_merge_returns, summarize_statistics, calculate_contract_risk, simulate_buy_and_hold, calculate_rolling_costs, calculate_combined_std
import numpy as np
from scipy.stats import skew

ImportError: cannot import name 'calculate_combined_std' from 'helper' (/home/jovyan/work/helper.py)

In [ ]:
# Initialize QuantBook
qb = QuantBook()

# Updated Instruments Dictionary
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.SP500EMini,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.MicroSP500EMini,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'ZN': {     # 10-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.015625,
        'minimum_fluctuation': 15.625,
        'spread': 0.015625,
        'commission': 2.50,
        'rolling_months': [6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Financials.Y10TreasuryNote,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'ZF': {  # 5-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.0078125,
        'minimum_fluctuation': 7.8125,
        'spread': 0.0078125,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Financials.Y5TreasuryNote,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'ZB': {  # 30-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.03125,
        'minimum_fluctuation': 31.25,
        'spread': 0.03125,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Financials.Y30TreasuryBond,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'ZW': {  # Wheat
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Grains.SRWWheat,
        'resolution': Resolution.Daily,
        'multiply_close': True
    },
    'CL': {  # Crude Oil
        'multiplier': 1000,
        'tick_value': 0.01,
        'minimum_fluctuation': 10.00,
        'spread': 0.01,
        'commission': 2.50,
        'rolling_months': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Energies.CrudeOilWTI,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'GC': {  # Gold
        'multiplier': 100,
        'tick_value': 0.1,
        'minimum_fluctuation': 10.00,
        'spread': 0.1,
        'commission': 2.50,
        'rolling_months': [2, 4, 6, 8, 10, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.Gold,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'HE': {  # Lean Hog
        'multiplier': 400,
        'tick_value': 0.00025,
        'minimum_fluctuation': 10.00,
        'spread': 0.025,
        'commission': 2.50,
        'rolling_months': [2, 4, 6, 7, 8, 10, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Meats.LeanHogs,
        'resolution': Resolution.Daily,
        'multiply_close': True
    },
    'GF': {  # Feeder Cattle
        'multiplier': 500,
        'tick_value': 0.00025,
        'minimum_fluctuation': 12.50,
        'spread': 0.00025,
        'commission': 2.50,
        'rolling_months': [1, 3, 4, 5, 8, 9, 10, 11],
        'subscription_type': 'Future',
        'subscription_details': Futures.Meats.FeederCattle,
        'resolution': Resolution.Daily,
        'multiply_close': True
    },
    'ZS': {  # Soy Bean
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'rolling_months': [1, 3, 5, 7, 8, 9, 11],
        'subscription_type': 'Future',
        'subscription_details': Futures.Grains.Soybeans,
        'resolution': Resolution.Daily,
        'multiply_close': True
    },
    'ZC': {  # Corn
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Grains.Corn,
        'resolution': Resolution.Daily,
        'multiply_close': True
    },
    'NG': {  # Natural Gas
        'multiplier': 10000,
        'tick_value': 0.001,
        'minimum_fluctuation': 10.00,
        'spread': 0.001,
        'commission': 2.50,
        'rolling_months': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Energies.NaturalGas,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'SI': {  # Silver
        'multiplier': 5000,
        'tick_value': 0.005,
        'minimum_fluctuation': 25.00,
        'spread': 0.005,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.Silver,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'HG': {  # Copper
        'multiplier': 25000,
        'tick_value': 0.0005,
        'minimum_fluctuation': 12.50,
        'spread': 0.0005,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.Copper,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'NQ': {  # Nasdaq
        'multiplier': 20,
        'tick_value': 0.25,
        'minimum_fluctuation': 5.00,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.NASDAQ100EMini,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'MGC': { # Micro gold
        'multiplier': 10,
        'tick_value': 0.1,
        'minimum_fluctuation': 1.00,
        'spread': 0.1,
        'commission': 2.50,
        'rolling_months': [2, 4, 6, 8, 10, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.MicroGold,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'SIL': { # Micro silver
        'multiplier': 1000,
        'tick_value': 0.005,
        'minimum_fluctuation': 5.00,
        'spread': 0.005,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.MicroSilver,
        'resolution': Resolution.Daily,
        'multiply_close': False
    },
    'USDMXN': { #forex exchange rate to mxn
        'subscription_type': 'Forex',
        'subscription_details': 'USDMXN',
        'resolution': Resolution.Daily,
        'multiplier': 1,
        'multiply_close': False
    }
}


In [ ]:
# # Define Instruments Dictionary
# instruments = {
#     'ES': {
#         'multiplier': 50,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.25,
#         'commission': 2.50,
#         'rolling_months': [3, 6, 9, 12],
#         'subscription_type': 'Future',
#         'subscription_details': Futures.Indices.SP500EMini,
#         'resolution': Resolution.Daily
#     },
#     'MES': {
#         'multiplier': 5,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 1.25,
#         'spread': 0.25,
#         'commission': 2.50,
#         'rolling_months': [3, 6, 9, 12],
#         'subscription_type': 'Future',
#         'subscription_details': Futures.Indices.MicroSP500EMini,
#         'resolution': Resolution.Daily
#     },
#     'USDMXN': {
#         'subscription_type': 'Forex',
#         'subscription_details': 'USDMXN',
#         'resolution': Resolution.Daily,
#         'multiplier': 1,

#     }
# }



In [96]:

# Add Subscriptions for Futures and Forex
subscriptions = {}
for symbol, details in instruments.items():
    if details['subscription_type'] == 'Future':
        future = qb.AddFuture(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = future.Symbol
    elif details['subscription_type'] == 'Forex':
        forex = qb.AddForex(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = forex.Symbol

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()
trading_days_per_year = 256
capital = 100000
target_risk = .20

# Request Historical Data
timeseries = {}
for symbol, sub_symbol in subscriptions.items():
    history = qb.History([sub_symbol], start_date, end_date, instruments[symbol]['resolution'])
    if 'close' in history.columns and 'volume' in history.columns:
        if instruments[symbol]['subscription_type'] == 'Future':
            history.reset_index(level=0, drop=True, inplace=True)
        if instruments[symbol]['multiply_close']:
            history['close'] *= 100
        timeseries[symbol] = history[['close', 'volume']]
    else:
        print(f"No 'close' or 'volume' column found for {symbol}")

# Clean up
del details
del end_date
del future
del forex
del history
del start_date
del sub_symbol
del symbol


No 'close' or 'volume' column found for USDMXN


In [97]:

# Subscribe to USDMXN forex pair
usdmxn = qb.AddForex("USDMXN", Resolution.Daily).Symbol

# Retrieve historical data
history = qb.History([usdmxn], 365, Resolution.Daily)

# Ensure the 'close' data is correctly extracted from the history DataFrame
fx_data = history.loc[usdmxn][['close']]

del history
del usdmxn

## Print the timeseries

In [111]:
#Print symbols and first 5 entires of historical_data
for symbol, data in timeseries.items():
    print(symbol)
    print(data.tail())


ES
                              close     volume  price_return  currency_return  \
symbol time                                                                     
ES 1S1 2024-05-22 20:00:00  5331.00  1079510.0        -13.00           -650.0   
       2024-05-23 20:00:00  5286.50  1722978.0        -44.50          -2225.0   
       2024-05-24 20:00:00  5320.75  1023163.0         34.25           1712.5   
       2024-05-27 20:00:00  5334.00    38251.0         13.25            662.5   
       2024-05-28 20:00:00  5334.00        0.0          0.00              0.0   

                            percentage_return  combined_std  
symbol time                                                  
ES 1S1 2024-05-22 20:00:00          -0.243263      0.083105  
       2024-05-23 20:00:00          -0.834740      0.080850  
       2024-05-24 20:00:00           0.647877      0.079016  
       2024-05-27 20:00:00           0.249025      0.077359  
       2024-05-28 20:00:00           0.000000      0.0755

###### Agrego volumen a timeseries con la suposición de que el volumen es una buena fuente de información. 

## Individual dataframe creation

In [99]:

for symbol, data in timeseries.items():
    globals()[symbol] = pd.DataFrame(data)

## Summary statistics
We can begin working with the aquired data to calculate some useful statistics, such as returns, standard deviations, skew and fat tail ratios. Use the helper function to process these statistics.

### Returns Time Series

In [100]:
# Creación de returns series

for symbol, price_df in timeseries.items():
    price_series = price_df['close']
    volume_series = price_df['volume']
    merged_df = calculate_and_merge_returns(symbol, price_series, volume_series, instruments)
    timeseries[symbol] = merged_df.loc[:, ['close', 'volume', 'price_return', 'currency_return', 'percentage_return']]

del price_df, price_series, volume_series, merged_df



### Summary Stats for each symbol

In [101]:
summary = summarize_statistics(timeseries, trading_days_per_year)


In [102]:
print("Summary Statistics for Each Symbol:")

for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])


Summary Statistics for Each Symbol:
ES:
{'mean_return': 0.038045126318084474, 'std_dev': 1.250117517944728, 'annualized_mean': 9.739552337429625, 'annualized_std_dev': 20.001880287115647, 'daily_sharpe_ratio': 0.03043323989302467, 'annualized_sharpe_ratio': 0.4869318382883947, 'monthly_skew': -14.351501476816757, 'lower_fat_tail_ratio': 2.5671774962213623, 'higher_fat_tail_ratio': 1.7477877767954346}
MES:
{'mean_return': 0.05140635400430362, 'std_dev': 1.3006064204993424, 'annualized_mean': 13.160026625101727, 'annualized_std_dev': 20.809702727989478, 'daily_sharpe_ratio': 0.03952491175967527, 'annualized_sharpe_ratio': 0.6323985881548043, 'monthly_skew': -7.6168672475407595, 'lower_fat_tail_ratio': 2.124435391500839, 'higher_fat_tail_ratio': 1.420983295929312}
ZN:
{'mean_return': 0.008177633041837988, 'std_dev': 0.3844559667208213, 'annualized_mean': 2.093474058710525, 'annualized_std_dev': 6.15129546753314, 'daily_sharpe_ratio': 0.021270662311703187, 'annualized_sharpe_ratio': 0.3403

## Backtest buy and hold 1 contract. Return is a dataframe or time series.

In [103]:
# # Rolling costs per instrument. uncomment to compute

# rolling_costs_per_instrument = {}

# for symbol, data in timeseries.items():
#     instrument_details = instruments[symbol]
#     rolling_costs = calculate_rolling_costs(instrument_details, data)
#     rolling_costs_per_instrument[symbol] = rolling_costs

# print(rolling_costs_per_instrument)


In [104]:
# Your existing simulation code
buyhold_results = {}

for symbol, data in timeseries.items():
    print(f"Symbol: {symbol}, Start Date: {data.index[0][1]}, End Date: {data.index[-1][1]}")
    instrument_details = instruments[symbol]
        
    num_contracts = 1  # Specify the number of contracts as needed
    
    bnh_return, bnh_return_mxn = simulate_buy_and_hold(data, instrument_details, fx_data, num_contracts)
    buyhold_results[symbol] = {
        'bnh_return': bnh_return,
        'bnh_return_mxn': bnh_return_mxn
    }
print(buyhold_results)

Symbol: ES, Start Date: 2007-01-04 19:00:00, End Date: 2024-05-28 20:00:00
Symbol: MES, Start Date: 2019-05-07 20:00:00, End Date: 2024-05-28 20:00:00
Symbol: ZN, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: ZF, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: ZB, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: ZW, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: CL, Start Date: 2007-01-04 19:00:00, End Date: 2024-05-28 20:00:00
Symbol: GC, Start Date: 2012-11-01 20:00:00, End Date: 2024-05-28 20:00:00
Symbol: HE, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: GF, Start Date: 2007-01-09 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: ZS, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: ZC, Start Date: 2007-01-04 18:00:00, End Date: 2024-05-28 19:00:00
Symbol: NG, Start Date: 2007-01-04 19:00:00, End Date: 2024-05-28 20:00:00
Symbol: SI, Start Date: 

# Strategy 2: Risk measurement, risk target and capital


## Contract risk in base currency

Update summary dictionary to include the daily and annual contract risk 

risk in currency terms = notional exposure * std dev

notional exposure = last closing price * multiplier

In [105]:
# Update summary with contract risks

summary = calculate_contract_risk(summary, instruments, timeseries)
print("Summary Statistics for Each Symbol with Contract Risks:")
for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])

Summary Statistics for Each Symbol with Contract Risks:
ES:
{'mean_return': 0.038045126318084474, 'std_dev': 1.250117517944728, 'annualized_mean': 9.739552337429625, 'annualized_std_dev': 20.001880287115647, 'daily_sharpe_ratio': 0.03043323989302467, 'annualized_sharpe_ratio': 0.4869318382883947, 'monthly_skew': -14.351501476816757, 'lower_fat_tail_ratio': 2.5671774962213623, 'higher_fat_tail_ratio': 1.7477877767954346, 'daily_contract_risk': 3334.0634203585896, 'annualized_contract_risk': 53345.014725737434}
MES:
{'mean_return': 0.05140635400430362, 'std_dev': 1.3006064204993424, 'annualized_mean': 13.160026625101727, 'annualized_std_dev': 20.809702727989478, 'daily_sharpe_ratio': 0.03952491175967527, 'annualized_sharpe_ratio': 0.6323985881548043, 'monthly_skew': -7.6168672475407595, 'lower_fat_tail_ratio': 2.124435391500839, 'higher_fat_tail_ratio': 1.420983295929312, 'daily_contract_risk': 346.8554747669184, 'annualized_contract_risk': 5549.6875962706945}
ZN:
{'mean_return': 0.00817

In [106]:
# Compute target risk in currency
target_risk_in_usd = capital * target_risk

# Compute position size
def calculate_position_size(target_risk_in_usd, contract_risk, capital):
    return target_risk_in_usd / contract_risk

In [107]:
# Calculate position size for each symbol
position_sizes = {}
for symbol, stats in summary.items():
    position_size = calculate_position_size(target_risk_in_usd, stats['annualized_contract_risk'], capital)
    position_sizes[symbol] = position_size
# Print position sizes
print ("Position sizes to achieve target risk per instrument:")
for symbol, position_size in position_sizes.items():
    print(f"{symbol}: {position_size}")


Position sizes to achieve target risk per instrument:
ES: 0.3749178831953828
MES: 3.6038064581220204
ZN: 2.988886080815944
ZF: 1.2843307642065427
ZB: 1.5482686128181888
ZW: 1.7236935798345534
CL: 0.6308944824165285
GC: 0.5609674356027508
HE: 1.8335617861481066
GF: 0.9130227692111085
ZS: 1.3627109189591395
ZC: 3.0744180867652435
NG: 1.5072802486234147
SI: 0.45111630169662315
HG: 0.7512131144575345
NQ: 0.23489949473625207
MGC: 1.3528653190930484
SIL: 0.8182661855812502


# Strategy 3, rolling windows and combined long and short term risk metric

In [108]:
def calculate_short_term_ewma(data):
    ewma_32 = data['percentage_return'].ewm(span=32, adjust=False).mean()
    ewma_32_std = ewma_32.ewm(span=32, adjust=False).std()
    return ewma_32, ewma_32_std

def calculate_long_term_ewma(data):
    ewma_2560 = data['percentage_return'].ewm(span=2560, adjust=False).mean()
    ewma_2560_std = ewma_2560.ewm(span=2560, adjust=False).std()
    return ewma_2560, ewma_2560_std


In [109]:
def calculate_combined_std(timeseries):
    """
    Calculate the combined standard deviation of returns using 0.3*long_term_ewma + 0.7*short_term_ewma.
    Returns a dictionary with symbols as keys and combined standard deviation as values.
    """
    combined_std = {}
    for symbol, data in timeseries.items():
        ewma_32, ewma_32_std = calculate_short_term_ewma(data)
        ewma_2560, ewma_2560_std = calculate_long_term_ewma(data)
        combined_std[symbol] = 0.3 * ewma_2560_std + 0.7 * ewma_32_std
    return combined_std


In [110]:
# Calcular la desviación estándar combinada y agregarla a timeseries
returns_std = calculate_combined_std(timeseries)
for symbol, std_series in returns_std.items():
    timeseries[symbol]['combined_std'] = std_series
